<a href="https://colab.research.google.com/github/ars7/Time-Series-Project/blob/main/Replication_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> **DME**

In [ ]:
library(nlme)
library(dummies)
library(forecast)
library(spatstat)
library(mice)
source("Programa.R")


#_______________________________________________________
RM<-read.csv("PM.csv",header=T,sep=",")
pesos<-read.csv("pesosdist.csv",header=T,sep=",")
w<-pesos[,"CERRILLOS"] 
RM<-rowSums(t(w*t(RM[,3:13])))
RM<-RM[8761:32130]
#_______________________________________________________
  
dato<-read.csv("cerrillosM.csv",header=T,sep=",")
dato<-dato[,-1]
dim(dato)
dato<-dato[8761:32130,] #01-01-2012  100 hasta 31-08-2014 1800


#Setup

n<-23370-168 

k1<-1# horas de predicción
k2<-153*(24/k1) # número de dias 
k3<-1 #retardo covariables

position<-as.character(dato[,1])

PMotros<-RM[(169-k3):(n+168-k3)]

Temp<-dato[(169-k3):(n+168-k3),"T"]
Hum<-dato[(169-k3):(n+168-k3),"HR"]
VV<-dato[(169-k3):(n+168-k3),"VV"]
CO<-dato[(169-k3):(n+168-k3),"CO"]
DV<-dato[(169-k3):(n+168-k3),"DV"]
DV<-DV-315
DV<-cut(DV,c(-315,-225,-135,-45,45))
DV<-dummify(DV)
colnames(DV)<-c("N","E","S","O")

dato<-dato[,"PM25"] 
dias<-weekdays(as.POSIXlt(position, format = "%Y-%m-%d"))
thetahd2<-Fourier(n,4)
dias1<-dummy(dias[145:(n+144)])

PMhd<-dato[169:(n+168)]
PMh1d<-dato[168:(n+167)]
PMhd1<-dato[145:(n+144)]
PMhd7<-dato[1:n]
hour<-rep(c(1:24),floor(n/24)+1)
hour<-hour[1:n]
PMmax<-rep(0,n)
for( i in 2:n) PMmax[i]<-max(PMh1d[pmax(1,i-24):i])


PMhd1<-PMhd1*dias1
PMhd7<-PMhd7*thetahd2
VV<-VV*DV

PM25all<-as.data.frame(cbind(PMhd,PMhd1,PMhd7,PMh1d,PMmax,CO,Temp,Hum,VV,PMotros,hour)) 
A<-matrix(0,k1,k2)
P<-matrix(0,k1,k2)
for( j in 1:k2){
  
  
PM25<-PM25all[(1+k1*(j-1)):(n-k1*k2 +k1*(j-1)),]
hour<-PM25all[(1+k1*(j-1)):(n-k1*k2 +k1*(j-1)),"hour"]

reg<-as.formula(paste("PMhd ~ ", paste(names(PM25)[2:(dim(PM25)[2]-1)], collapse = " + "),"| hour"))
reg
fm<- lmList(reg, PM25)
ehd<-change(residuals(fm))
ehd1<-c(rep(0,24),ehd[1:(length(ehd)-24)])
ehd7<-c(rep(0,24*7),ehd[1:(length(ehd)-24*7)])

PM25<-as.data.frame(cbind(PM25[1:(dim(PM25)[2]-1)],ehd1,ehd7,hour))

reg<-as.formula(paste("PMhd ~ ", paste(names(PM25)[2:(dim(PM25)[2]-1)], collapse = " + "),"| hour"))
reg

fm<- lmList(reg, PM25,na.action = na.omit)
betas<-coef(fm)

error<-max(betas)


while(error>1e-09){
  
  error<-betas
  ehd<-change(residuals(fm))
  ehd1<-c(rep(0,24),ehd[1:(length(ehd)-24)])
  ehd7<-c(rep(0,24*7),ehd[1:(length(ehd)-24*7)])

  PM25<-as.data.frame(cbind(PM25[1:(dim(PM25)[2]-3)],ehd1,ehd7,hour))
  
  fm<- lmList(reg, PM25,na.action = na.omit)
  betas<-coef(fm)
  error<-max(abs(error-betas))
    
}


cat("iter:",j,"ajuste",ajuste(fm,PM25[,"PMhd"])[1:3],"\n")

PM25b<-PM25all[(n-k1*k2 +k1*(j-1) +1):n,] 
PM25b[,"PMh1d"]<-NA
PM25b[,"PMmax"]<-NA
PMh1d<-PM25all[(n-k1*k2 +k1*(j-1) +1 -24):(n-k1*k2 +k1*(j-1) +1),"PMh1d"]
PMmax<-max(PMh1d)
ehd1<-rep(NA,dim(PM25b)[1])
ehd7<-rep(NA,dim(PM25b)[1])

PM25b<-as.data.frame(cbind(PM25b,ehd7,ehd1))

ehd<-change(residuals(fm))
m<-length(ehd)
ehd1<-ehd[(m-24):(m-1)]
ehd7<-ehd[ (m-24*7):(m-24*6 -1)]

PM25b[1:24,"ehd1"]<-ehd1
PM25b[1:24,"ehd7"]<-ehd7
PM25b[1,"PMh1d"]<-tail(PMh1d,1)
PM25b[1,"PMmax"]<-PMmax

for ( i in 1:k1){
A[i,j]<-as.numeric(predict(fm,newdata=PM25b[i,]))
P[i,j]<-PM25b[i,"PMhd"]
PM25b[(i+1),"PMh1d"]<-A[i,j]
PMh1d<-c(PMh1d[-1],A[i,j])
PM25b[(i+1),"PMmax"]<-max(PMh1d)
}

}

P<-as.vector(P)
A<-as.vector(A)
write.csv(cbind(P,A),"Mcerrillos1401M2.csv")





> **SARIMAX**




In [ ]:

library(forecast) 
library(dummies)
library(spatstat)
source("Programa.R")

RM<-read.csv("PM.csv",header=T,sep=",")
pesos<-read.csv("pesosdist.csv",header=T,sep=",")
w<-pesos[,"CERRILLOS"]
RM<-rowSums(t(w*t(RM[,3:13])))
RM<-RM[8761:32130]
#_______________________________________________________


dato<-read.csv("cerrillosM.csv",header=T,sep=",")
dato<-dato[,-1]
dim(dato)

dato<-dato[8761:32130,] #01-01-2012  100 hasta 31-08-2014 1800

n<-23370-168 
k1<-1# horas de predicción
k2<-153*(24/k1) # número de dias 
k3<-1 #retardo covariables

position<-as.character(dato[,1])

PMotros<-RM[(169-k3):(n+168-k3)]

Temp<-dato[(169-k3):(n+168-k3),"T"]
Hum<-dato[(169-k3):(n+168-k3),"HR"]
VV<-dato[(169-k3):(n+168-k3),"VV"]
CO<-dato[(169-k3):(n+168-k3),"CO"]
DV<-dato[(169-k3):(n+168-k3),"DV"]
DV<-DV-315
DV<-cut(DV,c(-315,-225,-135,-45,45))
DV<-dummify(DV)
colnames(DV)<-c("N","E","S","O")

dato<-dato[,"PM25"] 

dias<-weekdays(as.POSIXlt(position, format = "%Y-%m-%d"))
thetahd2<-Fourier(n,4)
dias1<-dummy(dias[145:(n+144)])

PMhd<-dato[169:(n+168)]
PMh1d<-dato[168:(n+167)]
PMhd1<-dato[145:(n+144)]
PMhd7<-dato[1:n]
hour<-rep(c(1:24),floor(n/24)+1)
hour<-hour[1:n]
PMmax<-rep(0,n)
for( i in 2:n) PMmax[i]<-max(PMh1d[pmax(1,i-24):i])


PMhd1<-PMhd1*dias1
PMhd7<-PMhd7*thetahd2
VV<-VV*DV

PM25all<-as.data.frame(cbind(PMhd,PMhd1,PMhd7,PMh1d,PMmax,CO,Temp,Hum,VV,PMotros,hour)) 

A<-matrix(0,k1,k2)
P<-matrix(0,k1,k2)

for( j in 1:k2){
  
  
PM25<-PM25all[(1+k1*(j-1)):(n-k1*k2 +k1*(j-1)),]

PMhd <-PM25[,"PMhd"]
PMhd1<-PM25[,c(2:8)]
PMhd7<-PM25[,c(9:17)]
PMmax<-PM25[,"PMmax"]
CO   <-PM25[,"CO"]
Temp <-PM25[,"Temp"]
Hum  <-PM25[,"Hum"]
N   <-PM25[,"N"]
S   <-PM25[,"S"]
E   <-PM25[,"E"]
O   <-PM25[,"O"]
PMotros <-PM25[,"PMotros"]

fit<- Arima(PMhd, order=c(1,0,1), seasonal=c(0,0,1),xreg=cbind(PMhd1,PMhd7,PMmax,CO,Temp,Hum,N,S,E,O,PMotros))

cat("iter:",j,"ajuste",accuracy(fit),"\n")

PM25b<-PM25all[(n-k1*k2 +k1*(j-1) +1):n,] 
PM25b[,"PMh1d"]<-NA
PM25b[,"PMmax"]<-NA
PMh1d<-PM25all[(n-k1*k2 +k1*(j-1) +1 -24):(n-k1*k2 +k1*(j-1) +1),"PMh1d"]
PMmax<-max(PMh1d)
PM25b[1,"PMh1d"]<-tail(PMh1d,1)
PM25b[1,"PMmax"]<-PMmax

for ( i in 1:k1){
  
  PMhd <-PM25b[i,"PMhd"]
  PMhd1<-PM25b[i,c(2:8)]
  PMhd7<-PM25b[i,c(9:17)]
  PMmax<-PM25b[i,"PMmax"]
  CO   <-PM25b[i,"CO"]
  Temp <-PM25b[i,"Temp"]
  Hum  <-PM25b[i,"Hum"]
  N   <-PM25b[i,"N"]
  S   <-PM25b[i,"S"]
  E   <-PM25b[i,"E"]
  O   <-PM25b[i,"O"]
  PMotros <-PM25b[i,"PMotros"] 
  
  A[i,j]<-as.numeric(forecast(fit,h=1,xreg=cbind(PMhd1,PMhd7,PMmax,CO,Temp,Hum,N,S,E,O,PMotros))$mean)
  P[i,j]<-PMhd 
  PM25b[(i+1),"PMh1d"]<-A[i,j]
  PMh1d<-c(PMh1d[-1],A[i,j])
  PM25b[(i+1),"PMmax"]<-max(PMh1d)
}

}


P<-as.vector(P)
A<-as.vector(A)

write.csv(cbind(P,A),"Acerrillos1401.csv")



> **ANN**

In [ ]:
library(forecast) 
library(dummies)
library(spatstat)
library(nnet)
library(caret)
source("Programa.R")
# Datos

dato<-read.csv("cerrillosM.csv",header=T,sep=",")
dato<-dato[,-1]
dim(dato)

dato<-dato[8761:32130,] #01-01-2012 100  hasta 31-08-2014 1800


#Setup

n<-23370-168 #19698 31-03-2014 1800 (153*24)
#23370 31-08-2014 1800 


k1<-1# horas de predicción
k2<-153*(24/k1) # número de dias 
k3<-1 #retardo covariables

position<-as.character(dato[,1])
dias<-weekdays(as.POSIXlt(position, format = "%Y-%m-%d"))
dias[dias=="lunes"]<-1
dias[dias=="martes"]<-2
dias[dias=="miércoles"]<-3
dias[dias=="jueves"]<-4
dias[dias=="viernes"]<-5
dias[dias=="sábado"]<-6
dias[dias=="domingo"]<-7
dias<-as.numeric(dias)
Temp<-dato[(169-k3):(n+168-k3),"T"]
Hum<-dato[(169-k3):(n+168-k3),"HR"]
VV<-dato[(169-k3):(n+168-k3),"VV"]
CO<-dato[(169-k3):(n+168-k3),"CO"]
DV<-dato[(169-k3):(n+168-k3),"DV"]
DV<-DV-315
DV<-cut(DV,c(-315,-225,-135,-45,45))
DV<-dummify(DV)
colnames(DV)<-c("N","E","S","O")

dato<-dato[,"PM25"] #168 lag de 7 dias 24*7

dias<-dias[145:(n+144)]
PMhd<-dato[169:(n+168)]
PMh1d<-dato[168:(n+167)]
PMhd1<-dato[145:(n+144)]
PMhd7<-dato[1:n]
hour<-rep(c(1:24),floor(n/24)+1)
hour<-hour[1:n]
PMmax<-rep(0,n)
for( i in 2:n) PMmax[i]<-max(PMh1d[pmax(1,i-24):i])

VV<-VV*DV

PM25all<-as.data.frame(cbind(PMhd,PMhd1,PMhd7,PMh1d,PMmax,CO,Temp,Hum,VV,dias,hour))
max.PM<-apply(PM25all,2,max)
PM25all<-t(t(PM25all)/max.PM)


A<-matrix(0,k1,k2)
P<-matrix(0,k1,k2)


for( j in 1:k2){
  
  
  PM25<-PM25all[(1+k1*(j-1)):(n-k1*k2 +k1*(j-1)),]
  
  set.seed(123)
  nnet.fit <- nnet(PMhd~., data=PM25, size=8,decay = 0, maxit = 20000,linout=T,trace=F) 
    nnet.predict <- predict(nnet.fit)
  
  res<-(PM25[,"PMhd"]-nnet.predict )*max.PM["PMhd"]

   cat("iter:",j,"ajuste",ajusteNN(res),"\n")

  PM25b<-PM25all[(n-k1*k2 +k1*(j-1) +1):n,] 
  PM25b[,"PMh1d"]<-NA
  PM25b[,"PMmax"]<-NA
  PMh1d<-PM25all[(n-k1*k2 +k1*(j-1) +1 -24):(n-k1*k2 +k1*(j-1) +1),"PMh1d"]
  PMmax<-max(PMh1d)
  PM25b[1,"PMh1d"]<-tail(PMh1d,1)
  PM25b[1,"PMmax"]<-PMmax
  special<-PMh1d
  
  for ( i in 1:k1){
    
    PMhd <-PM25b[i,"PMhd"]
    PMhd1<-PM25b[i,"PMhd1"]
    PMhd7<-PM25b[i,"PMhd7"]
    PMh1d<-PM25b[i,"PMh1d"]
    PMmax<-PM25b[i,"PMmax"]
    CO   <-PM25b[i,"CO"]
    Temp <-PM25b[i,"Temp"]
    Hum  <-PM25b[i,"Hum"]
    N   <-PM25b[i,"N"]
    S   <-PM25b[i,"S"]
    E   <-PM25b[i,"E"]
    O   <-PM25b[i,"O"]
    hour<-PM25b[i,"hour"]
    dias<-PM25b[i,"dias"]
    
    A[i,j]<-as.numeric(predict(nnet.fit,newdata=cbind(PMhd,PMhd1,PMhd7,PMh1d,PMmax,CO,Temp,Hum,N,S,E,O,dias,hour)))
    P[i,j]<-PMhd 
    PM25b[(i+1),"PMh1d"]<-A[i,j]
    special<-c(special[-1],A[i,j])
    PM25b[(i+1),"PMmax"]<-max(special)
  }
  
  
}

P<-as.vector(P)*max.PM["PMhd"]
A<-as.vector(A)*max.PM["PMhd"]

write.csv(cbind(P,A),"Ncerrillos1401.csv")
